# Regresssion with outliers

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
%%capture
%load_ext jupyter_probcomp.magics
%matplotlib inline

Load the read infer predict layer (ripl) and set venture's random seed to 1 to
ensure reproducibility.

In [ ]:
%ripl --seed 42 --plugins resources/helper_plugins.py

## 1. Generate Synthetic data

Save the synthetic data to file.

## 2. Venture model component

In [ ]:
%%venturescript
// MODEL
assume a = normal(0, 1) #continuous:0;
assume b = normal(0, 1) #continuous:1;
assume model_noise =  -log_logistic(log_odds_uniform() #continuous:2);
assume outlier_noise = 100;
assume outlier_probability = 0.01;

assume model = (x) -> { 
    if (bernoulli(outlier_probability) #discrete:x) {
        normal(0, outlier_noise)
    } else {
        normal(a * x + b, model_noise)
    }
};

## 3. Venture observation component

In [ ]:
%%venturescript
// OBSERVATIONS
// First, generate synthetic data.
define true_a = 2.5;
define true_b = 3;

define N = 20;

define max_xs = 13;
define regular_xs = mapv((_) -> {uniform_continuous(1, max_xs)}, arange(N));
define regular_ys = mapv((x) -> {true_a * x + true_b}, regular_xs);

define outliers_xs = [4, 6 , 8, 10];
define outliers_ys = [30, -12, -15, 37];

define data_xs = append(outliers_xs, regular_xs);
define data_ys = append(outliers_ys, regular_ys);
// Second observe synthetically generated data.
for_each(arange(size(data_xs)),
    (i) -> {
    observe model(${data_xs[i]}) = data_ys[i]
});


## 4. Venture infererence components

In [ ]:
%%venturescript
// INFERENCE
define single_site_mh  = () -> {
    mh(default, one, 1)
};
define lbfgs_with_gibbs = () -> {
    gibbs(quote(discrete), one, 1);
    lbfgs_optimize(quote(continuous), all)
};
define loop_explicitly_over_random_choices = () -> {
    for_each(data_xs,
        (x) -> {
            gibbs(quote(discrete), x, 1)
    });
    mh(quote(continuous), 0, 1);
    mh(quote(continuous), 1, 1);
    mh(quote(continuous), 2, 1)
};
define hamiltonian_monte_carlo_with_gibbs = () -> {
    gibbs(quote(discrete), one, 1);
    hmc(quote(continuous), one, 0.1, 1, 1)
};

In [ ]:
data_xs = %venturescript data_xs
data_ys = %venturescript data_ys
regular_xs = %venturescript regular_xs
regular_ys = %venturescript regular_ys
outliers_xs = %venturescript outliers_xs
outliers_ys = %venturescript outliers_ys

In [ ]:
path = 'resources/datasets/synthetic/lin-reg-outliers/'

In [ ]:
pd.DataFrame({'x': data_xs}).to_csv(    path + 'data-for-outlier-regs-x.csv',         index=False)
pd.DataFrame({'y': data_ys}).to_csv(    path + 'data-for-outlier-regs-y.csv',         index=False)
pd.DataFrame({'x': regular_xs}).to_csv( path + 'regular-data-for-outlier-regs-x.csv', index=False)
pd.DataFrame({'y': regular_ys}).to_csv( path + 'regular-data-for-outlier-regs-y.csv', index=False)
pd.DataFrame({'x': outliers_xs}).to_csv(path + 'outlier-data-for-outlier-regs-x.csv', index=False)
pd.DataFrame({'y': outliers_ys}).to_csv(path + 'outlier-data-for-outlier-regs-y.csv', index=False)

## 5. Execute inference programs and plot results

In [ ]:
def plot_n_traces(inf_prog, n):
    fig, ax = plt.subplots()
    for _ in range(n):
        plot_trace(inf_prog, ax)
    ax.scatter(outliers_xs, outliers_ys, label='Outlier data', color='red')    
    ax.scatter(regular_xs, regular_ys, label='Regular data', color='green')
    ax.plot([], [], color='blue', label='5 samples')
    ax.legend(
        loc='upper center',
        bbox_to_anchor=(0.5, 1.12),
        ncol=3,
        fontsize=8.
    )
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_xlim([0, 15])
    ax.set_ylim([-20, 40])
    fig.tight_layout()
    fig.set_size_inches(4, 3)
    #fig.savefig('results/linear-regression-with-outliers/pdf-files/%s.pdf' % (inf_prog,), bbox_inches='tight',)
    ax.set_title('%s: trace after %d steps'  % (inf_prog, number_frames,), y=1.11);
    #fig.savefig('results/linear-regression-with-outliers//%s.png' % (inf_prog,), bbox_inches='tight',)

In [ ]:
number_frames = 5

In [ ]:
T = {
    'single_site_mh': 27,
    'lbfgs_with_gibbs': 7,
    'loop_explicitly_over_random_choices': 1,
    'hamiltonian_monte_carlo_with_gibbs': 14,

}

In [ ]:
def plot_trace(inf_prog, ax):
    %venturescript reset_to_prior;
    for _ in range(number_frames * T[inf_prog]):
        %venturescript {inf_prog}();
    a = %venturescript sample(a)
    b = %venturescript sample(b)
    all_x_values= np.linspace(1, 13, 100)
    all_f_values = a * all_x_values + b
    ax.plot(all_x_values, all_f_values, color='blue')
    return ax

In [ ]:
def run_inference(inf_prog):
     plot_n_traces(inf_prog, 5)

In [ ]:
inf_progs = [
    'single_site_mh',
    'lbfgs_with_gibbs',
    'loop_explicitly_over_random_choices',
    'hamiltonian_monte_carlo_with_gibbs'
]

In [ ]:
scores = {}
for inf_prog in inf_progs:
    run_inference(inf_prog)